In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('C:/Users/Pc_User/Downloads/Raw_Data_HFARM.csv').drop('Unnamed: 0', axis=1)

## Some info about the data for a future double check

In [2]:
df.head(5)

,_id,mall,date,customer,desc,net_am,n_unit
0,1001,2,2016-01-14T20:07:00.000+0000,77021708271,caramelos s/azucar,1.00,1
1,1002,1,2016-01-14T15:25:00.000+0000,77021708271,tosta variada,3.00,1
2,1002,1,2016-01-14T15:25:00.000+0000,77021708271,baguette tortilla,1.00,1
3,1003,2,2016-01-14T20:07:00.000+0000,77021708271,queso tierno mezcl,2.83,1
4,1003,2,2016-01-14T20:07:00.000+0000,77021708271,guisantes muy fino,1.65,1


In [3]:
print(f"The total number of customers is {len(np.unique(df['customer']))}")
print(f"The total number of tickets is {len(np.unique(df['_id']))}")
print(f"The total number of non-unique bought items is {len(df)}")
print(f"The average number of items per ticket is {len(df) / len(np.unique(df['_id']))}")

The total number of customers is 66058
The total number of tickets is 586479
The total number of non-unique bought items is 6848976
The average number of items per ticket is 11.678126582537482


## Time engineering 

In [4]:
df['date'] = pd.to_datetime(df['date'], format= "%Y-%m-%dT%H:%M:%S.000+0000")
df['month'] = df['date'].dt.month
df['day_of_year'] = df['date'].dt.dayofyear

In [5]:
df.head(5)

,_id,mall,date,customer,desc,net_am,n_unit,month,day_of_year
0,1001,2,2016-01-14 20:07:00,77021708271,caramelos s/azucar,1.00,1,1,14
1,1002,1,2016-01-14 15:25:00,77021708271,tosta variada,3.00,1,1,14
2,1002,1,2016-01-14 15:25:00,77021708271,baguette tortilla,1.00,1,1,14
3,1003,2,2016-01-14 20:07:00,77021708271,queso tierno mezcl,2.83,1,1,14
4,1003,2,2016-01-14 20:07:00,77021708271,guisantes muy fino,1.65,1,1,14


## Frequency of all the items

In [6]:
df['desc'].value_counts()

bolsa carrefour         144226
platano 1a bolsa         41353
pan pistola/barra        28252
freson tarrina 500       21746
panecillos surtidos      21236
baguette carrefour       21223
zanahoria bolsa 1kg      17985
pimiento italiano        17298
cerveza mahou clas       16746
calabacin campo          15754
freson caja 2 kg         15501
limon carrefour 750g     15452
cebolla carrefour        15228
pimiento rojo camp       15172
brocoli filmado          14812
atun claro aceite        14749
banana granel            14567
patatas fritas lay       13885
pera conferencia         13598
manzana golden           13142
cerveza mahou 5 *        12797
barras pan 3 unida       12639
naranja zumo carre       11825
puerro manojo 3 u        11714
lechuga iceberg          11325
azucar bolsa papel       11182
judias verdes plan       11168
esparrago verde mano     10707
huevos camperos 12       10549
revistas 4% iva          10404
                         ...  
sec.pelo.rem d5216           1
venta de

In [7]:
df["desc"] = df["desc"].apply(lambda x : x.strip() if x != 'ropa de lulu, la ' else x) #excel is not stripping this string, so i'll leave it not stripped

## Translate items in english

In [8]:
#save them in a csv, to translate them in english with excel, and reupload them
not_translated_directory = "C:/Users/Pc_User/Desktop/H-Farm/translation.csv" #change it with yours
df['desc'].value_counts().to_csv(not_translated_directory)

In [9]:
#reupload
translated_directory = "C:/Users/Pc_User/Desktop/H-Farm/Item Translation 08-10-2020.csv" #change it with yours
translation = pd.read_csv(translated_directory).drop('Count', axis=1).drop_duplicates(subset = "Item") #excel creates double translations for some string

In [10]:
#merge the translation
data = df.merge(translation, left_on='desc', right_on = 'Item', how='left').drop('Item', axis=1)
data.head(5)

,_id,mall,date,customer,desc,net_am,n_unit,month,day_of_year,Translation
0,1001,2,2016-01-14 20:07:00,77021708271,caramelos s/azucar,1.00,1,1,14,s candy / sugar
1,1002,1,2016-01-14 15:25:00,77021708271,tosta variada,3.00,1,1,14,tosta varied
2,1002,1,2016-01-14 15:25:00,77021708271,baguette tortilla,1.00,1,1,14,omelet baguette
3,1003,2,2016-01-14 20:07:00,77021708271,queso tierno mezcl,2.83,1,1,14,soft cheese MIXER
4,1003,2,2016-01-14 20:07:00,77021708271,guisantes muy fino,1.65,1,1,14,Peas very fine


## check everything is ok

In [11]:
#this should run fine
assert len(np.unique(translation[~translation["Item"].isin(list(df['desc']))]["Item"])) == 0
assert len(np.unique(df[~df["desc"].isin(list(translation["Item"]))]["desc"])) == 0
assert data.isnull().sum().sum() == 0 
assert len(data) == len(df) 
print("Everything ok")

Everything ok


## check new table is the same as before

In [12]:
print(f"The total number of customers is {len(np.unique(data['customer']))}")
print(f"The total number of tickets is {len(np.unique(data['_id']))}")
print(f"The total number of non-unique bought items is {len(data)}")
print(f"The average number of items per ticket is {len(data) / len(np.unique(data['_id']))}")

The total number of customers is 66058
The total number of tickets is 586479
The total number of non-unique bought items is 6848976
The average number of items per ticket is 11.678126582537482


## save dataset  for next notebook

In [13]:
data.to_csv("C:/Users/Pc_User/Desktop/H-Farm/dataset_translation_datetime_v1.csv")